<a href="https://colab.research.google.com/github/ShreyJ1729/EVA6-TSAI/blob/main/05-CodingDrillDown/Iteration_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.convblock1 = nn.Sequential(
            nn.Conv2d(1, 11, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.Conv2d(11, 11, 3),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.Conv2d(11, 11, 3),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.MaxPool2d(2, 2)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv2d(11, 11, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.Conv2d(11, 11, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.Conv2d(11, 11, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.MaxPool2d(2, 2)
        )
        
        self.convblock3 = nn.Sequential(
            nn.Conv2d(11, 11, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.Conv2d(11, 11, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(11),

            nn.Conv2d(11, 11, 3, padding=1),
            
            nn.AvgPool2d(6)
        )

        self.fcblock = nn.Linear(11, 11)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = x.view(-1, 11)
        x = self.fcblock(x)
        return F.log_softmax(x, dim=1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
            Conv2d-4           [-1, 10, 26, 26]             910
              ReLU-5           [-1, 10, 26, 26]               0
       BatchNorm2d-6           [-1, 10, 26, 26]              20
            Conv2d-7           [-1, 10, 24, 24]             910
              ReLU-8           [-1, 10, 24, 24]               0
       BatchNorm2d-9           [-1, 10, 24, 24]              20
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 10, 12, 12]             910
             ReLU-12           [-1, 10, 12, 12]               0
      BatchNorm2d-13           [-1, 10, 12, 12]              20
           Conv2d-14           [-1, 10,

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, optimizer, epoch):
    epoch_losses = []
    model.train()
    pbar = tqdm(train_loader, leave=False)
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        tqdm.write(f'\r[Epoch {epoch}] loss={round(sum(epoch_losses)/len(epoch_losses), 4)}', end="")
    tqdm.write('\r[Epoch {}]\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(epoch, 
        round(sum(epoch_losses)/len(epoch_losses), 4), correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    tqdm.write('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

[Epoch 0]
Train set: Average loss: 0.4671, Accuracy: 50944/60000 (84.91%)
Test set: Average loss: 0.1072, Accuracy: 9669/10000 (96.69%)



[Epoch 1]
Train set: Average loss: 0.0904, Accuracy: 58434/60000 (97.39%)
Test set: Average loss: 0.1001, Accuracy: 9697/10000 (96.97%)



[Epoch 2]
Train set: Average loss: 0.0654, Accuracy: 58840/60000 (98.07%)
Test set: Average loss: 0.0502, Accuracy: 9859/10000 (98.59%)



[Epoch 3]
Train set: Average loss: 0.0506, Accuracy: 59066/60000 (98.44%)
Test set: Average loss: 0.0404, Accuracy: 9872/10000 (98.72%)



[Epoch 4]
Train set: Average loss: 0.0419, Accuracy: 59239/60000 (98.73%)
Test set: Average loss: 0.0437, Accuracy: 9867/10000 (98.67%)



...

[Epoch 10]
Train set: Average loss: 0.0236, Accuracy: 59550/60000 (99.25%)
Test set: Average loss: 0.0324, Accuracy: 9892/10000 (98.92%)



[Epoch 11]
Train set: Average loss: 0.0230, Accuracy: 59573/60000 (99.29%)
Test set: Average loss: 0.0330, Accuracy: 9897/10000 (98.97%)



[Epoch 12]
Train set: Average loss: 0.0215, Accuracy: 59593/60000 (99.32%)
Test set: Average loss: 0.0284, Accuracy: 9916/10000 (99.16%)



[Epoch 13]
Train set: Average loss: 0.0187, Accuracy: 59655/60000 (99.42%)
Test set: Average loss: 0.0287, Accuracy: 9913/10000 (99.13%)



[Epoch 14]
Train set: Average loss: 0.0182, Accuracy: 59662/60000 (99.44%)
Test set: Average loss: 0.0327, Accuracy: 9892/10000 (98.92%)



Target:
- \>99% Test Accuracy
- Under 10k parameters

Results:
- 99.16% accuracy (didn't stick till the end)
- ~7.5k parameters
- accuracy jumping around near the end
- overfitting in the last few epochs

Analysis:
- need an LR-Scheduler to optimize lr
- need dropout for regularizing overfitting